In [2]:
! pip3 install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
import glob
import os
import boto3
import socket
from botocore.handlers import disable_signing
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings("ignore")

In [4]:
if socket.gethostname() == 'Rohits-MacBook-Pro.local':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/data_parquet/'
else: 
    rootdir = '' # Enter your hone dir here

In [5]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
files_with_size = [ file_path for file_path in list_of_files ]

dfs_parquet = {}
for file_path in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')

Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet


In [6]:
# Dropping extra data for Thuesday-20-02-2018 file
dfs_parquet['Thuesday-20-02-2018'].drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP'], axis=1, inplace=True)

In [7]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 80)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)


In [8]:
cat_cols = ['Dst Port']

In [9]:
def transformations(df_list):
    df_temp = pd.concat(df_list)
    df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'])
    df_temp['Date'] = pd.to_datetime(df_temp['Timestamp']).dt.date
    df_temp['TS_relative'] = (df_temp['Timestamp'].astype(int) - 
                             pd.to_datetime(df_temp['Date']).astype(int))/ 10**9
    df_temp = df_temp.drop(['Timestamp'], axis = 1)
    df_temp = df_temp.drop(['Date'], axis = 1)
    df_temp.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_temp.fillna(0, inplace=True)
    df_temp[cat_cols] = df_temp[cat_cols].astype('category')
    X_temp = df_temp.drop(['is_allowed'], axis = 1)
    y_temp = df_temp['is_allowed']*1 
    
    return X_temp, y_temp

In [10]:
def scale_data(df):
    stdsc = MinMaxScaler()
    stdsc.fit(df)
    return stdsc.transform(df)

### Creating Train and Test Data sets

In [11]:
X_train, y_train = transformations([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Friday-16-02-2018'], 
                      dfs_parquet['Friday-23-02-2018'],
                      dfs_parquet['Thursday-01-03-2018'],
                      dfs_parquet['Thursday-15-02-2018'],
                      dfs_parquet['Thursday-22-02-2018'],
                      dfs_parquet['Thuesday-20-02-2018'],
                      dfs_parquet['Wednesday-14-02-2018'],
                      dfs_parquet['Wednesday-21-02-2018']
                     ])

In [12]:
X_train_scaled = scale_data(X_train)
X_train.shape

(15619872, 79)

In [13]:
X_test, y_test = transformations([dfs_parquet['Wednesday-28-02-2018']])

In [14]:
X_test_scaled = scale_data(X_test)
X_test.shape

(613071, 79)

### Preparing data for Pytorch by converting Dataframes into Tensors

In [15]:
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape

torch.Size([15619872, 79])

In [16]:
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([15619872])

In [17]:
x_for_nn_test =  torch.from_numpy(X_test_scaled).float()
x_for_nn_test.shape

torch.Size([613071, 79])

In [18]:
y_for_nn_test = torch.from_numpy(y_test.to_numpy())
y_for_nn_test.shape

torch.Size([613071])

### Model Setup

- Layers:
1. Linear input=79 out=128
2. BayesLinear input=128 out=128
3. BayesLinear input=120 out=2 (Benign or Not Benign)

- Activation Function = ReLU

- Loss Function Hybrid:
 Cross Entropy and Kullback-Leibler Divergence (KL divergence)

- Optimizer = Adam with Learning Rate = 0.01

In [22]:
model = nn.Sequential(
    nn.Linear(in_features=79, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=128),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=128, out_features=2),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

optimizer = optim.Adam(model.parameters(), lr=0.01)

### Training with Batches

In [20]:
batch_size = 1048576
train_data = TensorDataset(x_for_nn, y_for_nn)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=False)

In [27]:
kl_weight = 0.1
epochs = 10
for epoch in range(epochs):
    start_time = time.process_time()
    avg_loss = 0.
    counter = 0
    for x, y in train_loader:
        counter += 1
        pre = model(x)
        ce = ce_loss(pre, y)
        kl = kl_loss(model)
        cost = ce + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        avg_loss += cost.item()
        if epoch%1 == 0:
            print(f"Epoch {epoch}, Batch {counter}...... Average Loss for Epoch: {avg_loss/counter}")
            _, predicted = torch.max(pre.data, 1)
            print(f'F1: {f1_score(y, predicted)}')
            print(f'Accuracy: {accuracy_score(y, predicted)}')
            print(f'Precision: {precision_score(y, predicted)}')
            print(f'Recall: {recall_score(y, predicted)}')            
    current_time = time.process_time()

Epoch 0, Batch 1...... Average Loss for Epoch: 0.1412525773048401
F1: 0.9717333804160712
Accuracy: 0.9523639678955078
Precision: 0.9556840077071291
Recall: 0.9883310138100846
Epoch 0, Batch 2...... Average Loss for Epoch: 0.1421404480934143
F1: 0.9675239022260227
Accuracy: 0.945526123046875
Precision: 0.9552333981864334
Recall: 0.9801347999829513
Epoch 0, Batch 3...... Average Loss for Epoch: 0.1426272690296173
F1: 0.9686441997730759
Accuracy: 0.9475793838500977
Precision: 0.9596884311234294
Recall: 0.9777686925978176
Epoch 0, Batch 4...... Average Loss for Epoch: 0.14221831411123276
F1: 0.9722576067259769
Accuracy: 0.9532060623168945
Precision: 0.9555052748442218
Recall: 0.9896078395263505
Epoch 0, Batch 5...... Average Loss for Epoch: 0.14204095900058747
F1: 0.9704947654380045
Accuracy: 0.9503545761108398
Precision: 0.955532439934418
Recall: 0.9859331238944575
Epoch 0, Batch 6...... Average Loss for Epoch: 0.14153132339318594
F1: 0.9723808764498697
Accuracy: 0.9534177780151367
Precis

Recall: 0.9885491833753665
Epoch 3, Batch 3...... Average Loss for Epoch: 0.13656889398892721
F1: 0.9734179578122083
Accuracy: 0.9551849365234375
Precision: 0.9569978344313186
Recall: 0.990411389381621
Epoch 3, Batch 4...... Average Loss for Epoch: 0.14283661916851997
F1: 0.9642727854353493
Accuracy: 0.9389162063598633
Precision: 0.9352918580556502
Recall: 0.9951071477088788
Epoch 3, Batch 5...... Average Loss for Epoch: 0.14022853076457978
F1: 0.9734568637135185
Accuracy: 0.9554100036621094
Precision: 0.9596433084517809
Recall: 0.9876739045038632
Epoch 3, Batch 6...... Average Loss for Epoch: 0.13852368791898093
F1: 0.9746724494198324
Accuracy: 0.9574508666992188
Precision: 0.9611701466602027
Recall: 0.9885595118417977
Epoch 3, Batch 7...... Average Loss for Epoch: 0.14606884121894836
F1: 0.9468286216900006
Accuracy: 0.9158010482788086
Precision: 0.9923699392776438
Recall: 0.9052838145822273
Epoch 3, Batch 8...... Average Loss for Epoch: 0.14393611811101437
F1: 0.9777507524372392
Accu

Precision: 0.9776711312169446
Recall: 0.9802492818025205
Epoch 6, Batch 5...... Average Loss for Epoch: 0.12780059427022933
F1: 0.9829391288082866
Accuracy: 0.9716424942016602
Precision: 0.9799002678057659
Recall: 0.9859968966406365
Epoch 6, Batch 6...... Average Loss for Epoch: 0.12601666276653609
F1: 0.9789071325064753
Accuracy: 0.9647407531738281
Precision: 0.970848209486412
Recall: 0.9871009682040189
Epoch 6, Batch 7...... Average Loss for Epoch: 0.12530323650155747
F1: 0.9851538110367661
Accuracy: 0.9754371643066406
Precision: 0.9867748726043675
Recall: 0.9835380668536565
Epoch 6, Batch 8...... Average Loss for Epoch: 0.12401741091161966
F1: 0.9789489193489253
Accuracy: 0.9648456573486328
Precision: 0.9711963160292162
Recall: 0.9868262893933323
Epoch 6, Batch 9...... Average Loss for Epoch: 0.123111459116141
F1: 0.986149414507976
Accuracy: 0.9771823883056641
Precision: 0.9916131037686315
Recall: 0.9807456040862337
Epoch 6, Batch 10...... Average Loss for Epoch: 0.12323888763785362

F1: 0.9854336545589325
Accuracy: 0.97601318359375
Precision: 0.9915943664073026
Recall: 0.9793490222441189
Epoch 9, Batch 7...... Average Loss for Epoch: 0.11272499923195158
F1: 0.9810005709448826
Accuracy: 0.9681692123413086
Precision: 0.9709639325346386
Recall: 0.9912468695681184
Epoch 9, Batch 8...... Average Loss for Epoch: 0.11177975684404373
F1: 0.9812451012368363
Accuracy: 0.9685773849487305
Precision: 0.9713290164957887
Recall: 0.9913657364542586
Epoch 9, Batch 9...... Average Loss for Epoch: 0.11111328999201457
F1: 0.9808702786783433
Accuracy: 0.9679813385009766
Precision: 0.971212769150397
Recall: 0.990721781197018
Epoch 9, Batch 10...... Average Loss for Epoch: 0.1108691781759262
F1: 0.986597648430263
Accuracy: 0.9778957366943359
Precision: 0.9914834072879162
Recall: 0.9817598048241576
Epoch 9, Batch 11...... Average Loss for Epoch: 0.11092777347022836
F1: 0.9814276414967005
Accuracy: 0.9688997268676758
Precision: 0.970904604151173
Recall: 0.992181283674313
Epoch 9, Batch 12

In [28]:
pre_test = model(x_for_nn_test)
_, predicted_test = torch.max(pre_test.data, 1)
print(f'F1: {f1_score(y_for_nn_test, predicted_test)}')
print(f'Accuracy: {accuracy_score(y_for_nn_test, predicted_test)}')
print(f'Precision: {precision_score(y_for_nn_test, predicted_test)}')
print(f'Recall: {recall_score(y_for_nn_test, predicted_test)}')

F1: 0.9144171274281725
Accuracy: 0.8430866897961248
Precision: 0.8863109969043451
Recall: 0.9443642043366409
